In [1]:
import numpy as np

Класс логического процесса. 
- load - величина нагрузки;
- loading - время с запуска задачи;
- before - множество процессов-родителей;
- after - множество процессов-потомков;
- denial_time - время отказа, либо -1
- agent - агент, выполняющий процесс

In [2]:
# время отказа, либо -1
def denial_of_service(load, p=0.05):
    b=np.random.choice([0,1], p=[1-p,p])
    if b:
        return round(np.random.choice(np.arange(0, load, 0.1)), 2)
    return -1

class LP:
    def __init__(self, ind, load):
        self.ind=ind
        self.load = load
        self.loading = 0
        self.before = set()
        self.after = set()
        self.denial_time = denial_of_service(load)
        self.agent=None

    def possible_to_start(self):
        return all(map(lambda x:x.load<=x.loading, self.before))
    
    def __repr__(self):
        return f"({self.load}) LP{self.ind} i/o={len(self.before)}/{len(self.after)}"
    
    def __str__(self):
        return self.__repr__()


In [3]:
class Agent:
    def __init__(self, ind):
        self.ind=ind
        self.ended_tasks = 0
        self.cur_task = None
        self.tasks = list()
        self.neighbors = set()
    
    def total_load(self):
        return sum([i.load for i in self.tasks]) + (self.cur_task.load if self.cur_task else 0)
    
    def __repr__(self):
        return f"Agent{self.ind}, load={self.total_load()}, {len(self.neighbors)} neighbors"
    
    def __str__(self):
        return self.__repr__()

In [4]:
from numpy.random import choice

def gen_lp(n_layers=4, min_load=1, max_load=5):
    ind = 0
    arch = choice(5, n_layers)+1
    def gen(k):
        nonlocal ind
        lps = set()
        for _ in range(k):
            lps.add(LP(ind, np.random.randint(min_load, max_load+1)))
            ind += 1
        return lps
    def connect(la,lb):
        n = len(la)
        for i in lb:
            for j in choice(list(la), choice(n//2+1)+1):
                i.before.add(j)
                j.after.add(i)
        for i in la:
            if not i.after:
                t = choice(list(lb), choice(1)+1)
                for j in t:
                    i.after.add(j)
                    j.before.add(i)
    
    st = gen(arch[0])
    la = st
    min_time = max(la, key=lambda x:x.load).load
    # print("слой 0:\n",la)
    j = 1
    for i in arch[1:]:
        lb = gen(i)
        connect(la,lb)
        print(f"слой {j-1}:\n",la)
        j+=1
        la = lb
        min_time += max(la, key=lambda x:x.load).load
    print(f"слой {j-1}:\n",lb)

    return st, min_time


In [5]:
def gen_agents(n=4):
    agents = set()
    for i in range(n):
        agents.add(Agent(i))

    for a in agents:
        if not a.neighbors:
            t = choice(list(agents-set([a])), choice(n) + 1)
            for b in t:
                if a != b:
                    a.neighbors.add(b)
                    b.neighbors.add(a)
    print("Связность агентов")
    for a in agents:
        print(a.ind, ': '+','.join([str(i.ind) for i in a.neighbors]))
    return agents

In [6]:
def get_most_idle_agent(agents):
    n = len(agents)
    return sorted(agents, key=lambda x:(x.total_load(), n-len(x.neighbors)))[0]

def system(st, agents):
    visited = set()
    stack = list(st)
    max_time = 0
    while stack:
        i = stack.pop(0)
        if not (i in visited or i.agent):
            max_time += i.load
            ag = get_most_idle_agent(agents)
            ag.tasks.append(i)
            i.agent = ag
            visited.add(i)
            stack.extend(list(i.after))
    # print(visited)
    # for a in agents:
    #     print(a.total_load())
    
    dt = 0.1
    t = 0
    while visited:
        t += dt
        for a in agents:
            if a.tasks or a.cur_task:
                if not a.cur_task:
                    for i in range(len(a.tasks)):
                        if a.tasks[i].possible_to_start():
                            a.cur_task = a.tasks.pop(i)
                            break
                    else:
                        continue
                    # a.cur_task = a.tasks.pop(0)
                task = a.cur_task
                task.loading += dt
                if (task.denial_time != -1) and (task.denial_time <= task.loading):
                    ag1 = get_most_idle_agent(a.neighbors)
                    i = 0
                    for i in range(len(ag1.tasks)):
                        if not ag1.tasks[i].possible_to_start():
                            break
                    ag1.tasks.insert(i, task)
                    a.cur_task = None
                    task.loading=0
                    task.denial_time = -1
                    print(f"{t:.1f}. Отказ в обслуживании на агенте {a.ind} (Задача {task.ind}). Перенос на агента {ag1.ind}")
                elif task.loading >= task.load:
                    print(f"{t:.1f}: Задача {task.ind} завершена агентом {a.ind}")
                    a.ended_tasks += 1
                    visited.remove(task)
                    a.cur_task = None
    print("Максимальное время работы программы:", max_time)

In [36]:
st, min_time = gen_lp(10, 10, 100)

слой 0:
 {(54) LP1 i/o=0/2, (57) LP0 i/o=0/1, (68) LP2 i/o=0/2, (24) LP3 i/o=0/2}
слой 1:
 {(13) LP4 i/o=3/2, (86) LP6 i/o=1/2, (40) LP7 i/o=1/3, (14) LP5 i/o=2/1}
слой 2:
 {(45) LP9 i/o=2/2, (32) LP8 i/o=2/1, (75) LP10 i/o=2/1, (53) LP11 i/o=2/3}
слой 3:
 {(76) LP15 i/o=2/1, (44) LP13 i/o=2/1, (56) LP12 i/o=1/1, (33) LP16 i/o=1/1, (88) LP14 i/o=1/1}
слой 4:
 {(49) LP19 i/o=1/1, (26) LP17 i/o=2/1, (10) LP18 i/o=2/2}
слой 5:
 {(62) LP21 i/o=2/3, (25) LP20 i/o=2/4}
слой 6:
 {(83) LP24 i/o=1/1, (53) LP25 i/o=2/1, (65) LP22 i/o=1/1, (66) LP26 i/o=2/1, (55) LP23 i/o=1/1}
слой 7:
 {(83) LP27 i/o=5/2}
слой 8:
 {(11) LP28 i/o=1/2, (30) LP29 i/o=1/1}
слой 9:
 {(24) LP31 i/o=1/0, (62) LP30 i/o=2/0}


In [37]:
min_time

686

In [39]:
agents = gen_agents(10)

Связность агентов
8 : 6,7,4
0 : 4,7
7 : 8,0,4,2,5,9
4 : 8,0,7
1 : 9
2 : 7,3
5 : 7
9 : 1,7,3
6 : 8
3 : 2,9


In [40]:
system(st, agents)

24.0: Задача 3 завершена агентом 9
36.0. Отказ в обслуживании на агенте 8 (Задача 0). Перенос на агента 7
54.0: Задача 1 завершена агентом 7
68.0: Задача 2 завершена агентом 4
81.1: Задача 4 завершена агентом 0
82.0: Задача 5 завершена агентом 1
111.0: Задача 0 завершена агентом 7
140.0: Задача 6 завершена агентом 2
150.9: Задача 7 завершена агентом 3
182.9: Задача 8 завершена агентом 6
193.0: Задача 11 завершена агентом 1
195.9: Задача 9 завершена агентом 5
225.9: Задача 10 завершена агентом 0
226.0: Задача 16 завершена агентом 7
251.8: Задача 12 завершена агентом 6
269.0: Задача 15 завершена агентом 3
269.8: Задача 13 завершена агентом 9
274.9: Задача 19 завершена агентом 1
284.0: Задача 14 завершена агентом 5
295.0: Задача 17 завершена агентом 8
305.1: Задача 18 завершена агентом 8
330.0: Задача 20 завершена агентом 9
367.0: Задача 21 завершена агентом 4
395.0: Задача 22 завершена агентом 8
413.0: Задача 24 завершена агентом 6
420.0: Задача 25 завершена агентом 0
421.9: Задача 23 за

In [41]:
for a in agents:
    print(f"Агент {a.ind}, кол-во реш. задач: {a.ended_tasks}")

Агент 8, кол-во реш. задач: 3
Агент 0, кол-во реш. задач: 3
Агент 7, кол-во реш. задач: 4
Агент 4, кол-во реш. задач: 3
Агент 1, кол-во реш. задач: 3
Агент 2, кол-во реш. задач: 2
Агент 5, кол-во реш. задач: 2
Агент 9, кол-во реш. задач: 5
Агент 6, кол-во реш. задач: 3
Агент 3, кол-во реш. задач: 4


In [691]:
tasks = list(st)
vis = set(st)
while vis:
    a = vis.pop()
    # print(a)
    for i in a.after:
        if not i in vis:
            tasks.append(i)
            vis.add(i)

In [692]:
tasks[0].after

{(7) LP3 i/o=1/0, (7) LP4 i/o=2/0}